In [2]:
import torch 
import torch.nn as nn 
import numpy as np 
import matplotlib.pyplot as plt 
import time

I'm using my base anaconda environment for this notebook, as opposed to the python runtimes I usually use for my other notebooks.

# NumPy vs Torch

Numpy **arrays** and PyTorch **tensors** are similar in many ways. They both are n-dimensional arrays, and they both support a large number of operations. However, there are some important differences between them. 

In [3]:
n = np.linspace(0, 1, 5)
t = torch.linspace(0, 1, 5)

In [4]:
n

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [5]:
t

tensor([0.0000, 0.2500, 0.5000, 0.7500, 1.0000])

They can be resized very similarly. 

In [6]:
n = np.arange(48).reshape(3, 4, 4)
t = torch.arange(48).reshape(3, 4, 4)

They have the same broadcasting rules.

# General Broadcasting Rules

**NumPy** compares the shape of the two arrays element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when 

1. they are equal, or
2. one of them is 1

**Example**: The following arrays are compatible for broadcasting:

    Shape1: (1, 6, 4, 1, 7, 2)
    Shape2: (5, 6, 1, 3, 1, 2)

Notice the numbers in parentheses are not actual elements of the array, but rather the shape of the array (they're both 6-dimensional arrays). The trailing dimensions are compatible because they are equal. The next two dimensions are compatible because one of them is 1. 
Going forward and checking element-wise, the two arrays are compatible for broadcasting.
    

    
    

    
    

In [7]:
a = np.array([1, 2])
b = np.array([3, 4])
a * b

array([3, 8])

In [8]:
a = np.ones((6, 5))
b = np.arange(5).reshape((1, 5))

In [9]:
print(a.shape)
print(b.shape)

(6, 5)
(1, 5)


They're compatible! The trailing dimensions are equal, and the next two dimensions are compatible because one of them is 1.

In [10]:
a + b

array([[1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.]])

$b$ has been broadcasted to match the shape of $a$, notice how it has been added elementwise and for each of the 5 rows.

All these examples work the same way using PyTorch tensors.

**Example**: Scaling each of the color channels of an image by a different amount 

        Image  (3D array): 256 x 256 x 3
        Scale  (1D array):             3
        Result (3D array): 256 x 256 x 3

In [11]:
Image = torch.randn((256, 256, 3))
Scale = torch.tensor([0.5, 1.5, 1])

In [12]:
Result = Image * Scale

Each pixel has its own RGB value and the scale array is broadcasted by automatically "filling" the missing dimensions with 1, which is compatible with the 3D array, as we saw above.

In [13]:
Result

tensor([[[ 5.4193e-01, -1.4909e-01,  3.3715e-01],
         [ 2.0500e-01,  7.0912e-01, -3.8141e-01],
         [-1.3512e-01, -4.9861e-01, -7.1900e-01],
         ...,
         [-6.9954e-02,  8.6427e-01, -1.6852e-02],
         [-4.0529e-01,  1.6589e+00,  1.3983e+00],
         [ 1.6758e-01,  6.9921e-01, -8.7392e-01]],

        [[ 5.5269e-02,  7.1054e-01,  2.1175e-01],
         [ 5.6743e-01, -1.0897e+00, -1.5861e-01],
         [-6.7587e-01, -1.9006e+00,  1.8689e+00],
         ...,
         [ 2.5398e-01, -2.9557e+00, -6.9693e-01],
         [ 3.2977e-01, -3.0399e-01,  5.4555e-01],
         [ 4.7734e-01,  9.1462e-01, -1.9316e-01]],

        [[ 3.2972e-01,  3.1169e+00, -1.4797e+00],
         [ 2.7645e-01, -2.6284e+00,  4.2197e-02],
         [ 3.9353e-02, -1.1383e-01,  1.1930e+00],
         ...,
         [-1.3341e-02, -1.2807e+00, -2.9090e-01],
         [ 4.8910e-01, -2.0508e-03,  5.8567e-02],
         [ 2.6874e-01,  2.3481e+00, -3.0073e-01]],

        ...,

        [[-5.0779e-01, -1.5866e+00, -9

**Example**: Suppose you have an array of 2 images and you want to scale the color channels of each image by a slightly different amount. 

        Image   (4D array):  2 x 256 x 256 x 3
        Scale   (2D array):  2 x 1   x 1   x 3
        Results (4D array):  2 x 256 x 256 x 3

In [14]:
Images = torch.randn((2, 256, 256, 3))
Scales = torch.tensor([0.5, 1.5, 1, 1.5, 1, 0.5]).reshape((2, 1, 1, 3))

In [15]:
Results = Images * Scales 

In [16]:
Results

tensor([[[[ 0.2608, -0.1968, -1.2438],
          [ 0.2564,  0.8337,  1.7528],
          [-0.2752, -3.8720,  0.1174],
          ...,
          [-0.9657,  1.7531,  0.2916],
          [ 0.9899,  0.3650,  0.5422],
          [ 0.5137, -1.1284, -0.4637]],

         [[-0.2700, -1.0392, -0.6198],
          [ 0.2428,  0.5081,  0.5560],
          [-0.4982, -0.0401,  0.1227],
          ...,
          [-0.6139, -1.1468,  0.4519],
          [-0.3639, -0.2186,  0.2458],
          [ 0.2020, -0.1111,  1.2536]],

         [[ 0.1925,  0.0419,  0.7937],
          [ 0.1874, -0.4274,  1.5424],
          [ 0.2707,  0.8403,  0.7752],
          ...,
          [-0.7021, -2.3563, -1.1848],
          [-0.3176, -0.6505,  1.6346],
          [-0.2568,  0.5206,  0.8773]],

         ...,

         [[-0.1045, -1.0587, -0.6398],
          [ 0.1956, -1.0350, -0.3277],
          [-0.3620,  0.2435,  0.2397],
          ...,
          [ 0.9061, -0.9410,  0.7956],
          [-0.6431, -2.4170, -1.1437],
          [ 0.4827,  0

Notice that in this scenario, the array would not be broadcasted correctly, as *PyTorch* would fill all the dimensions with 1s, whereas we needed to have the first dimension as 2.

In this kind of situations, we utilized the `reshape` subroutine to serve our needs. 

It is always important to spend some time thinking about the shapes of the arrays you are working with, and to make sure that the operations you are performing are valid.

# Operations Across Dimensions

This is a fundamental aspect of **PyTorch**. Let's see first a one dimensional tensor, which is trivial.

In [17]:
t = torch.tensor([0.5, 1, 3, 4])
torch.mean(t), torch.std(t), torch.max(t), torch.min(t)

(tensor(2.1250), tensor(1.6520), tensor(4.), tensor(0.5000))

Suppose we have a 2D tensor instead and we want to compute the mean of each column. This means taking the mean of each dimension, so it's an operation across dimensions.

In [18]:
t = torch.arange(20, dtype=float).reshape(5, 4)
t, torch.mean(t, axis=0)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]], dtype=torch.float64),
 tensor([ 8.,  9., 10., 11.], dtype=torch.float64))

This can be of course generalized for higher dimensional tensors.

In [19]:
t = torch.randn(5, 256, 256, 3)

This could represent a batch of 5 images, with resolution of 256 x 256 pixels, and 3 color channels per pixel.

Mean across all the images, so we get a single image with the mean of each pixel across all the images, including the color values.

In [20]:
torch.mean(t, axis=0), torch.mean(t, axis=0).shape

(tensor([[[-0.3562, -0.4056,  0.2154],
          [ 0.5333, -0.0079,  0.2167],
          [-0.5909,  0.4377, -0.1056],
          ...,
          [-0.1916,  0.2542, -0.4962],
          [ 0.0158,  0.0699,  0.1010],
          [ 0.3502,  0.4203, -0.0420]],
 
         [[ 0.0630, -0.1332, -0.8394],
          [ 0.6533,  0.2290, -0.5519],
          [ 0.7248, -0.5055,  0.0945],
          ...,
          [-0.0319,  0.0365,  0.2041],
          [-0.3483,  0.1173,  0.2313],
          [ 0.1733, -0.2464, -0.7887]],
 
         [[-0.1424,  0.1337, -0.3838],
          [-0.8562,  0.0780,  0.3785],
          [ 0.3333, -1.1031, -0.0909],
          ...,
          [-0.0325, -0.8204, -0.2991],
          [-0.4003,  0.9888,  0.0232],
          [ 0.2096, -0.5949, -0.6839]],
 
         ...,
 
         [[ 0.8043, -0.2688, -0.1386],
          [ 0.5974,  0.0413,  0.4535],
          [-0.4871,  0.7350,  0.0976],
          ...,
          [ 0.5585, -0.3826, -0.6759],
          [-0.3808,  0.4308, -0.5710],
          [ 0.3927

Mean across the color channels only:

In [21]:
torch.mean(t, axis=-1), torch.mean(t, axis=-1).shape

(tensor([[[ 0.5468,  0.1881,  0.4608,  ..., -0.2062,  0.1819,  0.3658],
          [ 0.3465,  0.0852,  0.3815,  ...,  0.1111, -0.0526, -0.0725],
          [-0.2049,  0.8760, -0.5861,  ..., -0.0394, -0.0922, -0.8019],
          ...,
          [-0.0092, -0.0455,  0.4595,  ..., -0.5586, -0.2606,  0.1232],
          [-0.1643,  0.2186,  0.2978,  ..., -0.7592, -0.5829, -0.1258],
          [ 0.0830, -0.7184,  0.0985,  ..., -0.8119, -0.0943,  0.3497]],
 
         [[-0.6953, -0.0172, -0.7987,  ..., -0.6359,  0.7798, -0.0479],
          [ 0.0264,  0.6073, -0.0743,  ..., -0.1566, -0.0672, -0.4779],
          [ 0.2467,  0.3761, -0.2785,  ..., -0.7459,  0.6505, -0.4954],
          ...,
          [ 0.4924,  0.3945, -0.2559,  ..., -0.1754,  0.7891, -0.0143],
          [ 0.1805,  0.0310, -0.1723,  ...,  0.8701,  0.5645, -0.7392],
          [ 0.0541,  0.7855,  0.0832,  ...,  0.5471,  0.7042, -0.6264]],
 
         [[-0.4218,  0.1707,  0.1059,  ...,  0.0957, -0.1982,  0.4580],
          [-0.3592,  0.1770,

Take only the maximum color channel values and the corresponging indices:

+ This is done all the time in image segmentation models, where we can isolate the pixels that belong to a certain class of object for which an algorithm has been trained, say a car

In [22]:
values, indices = torch.max(t, axis=-1)

values, indices

(tensor([[[ 1.0671e+00,  6.8944e-01,  9.5293e-01,  ...,  1.3474e+00,
            1.0383e+00,  7.7519e-01],
          [ 9.4600e-01,  1.7104e+00,  9.9561e-01,  ...,  6.1847e-01,
            6.1800e-01,  1.5198e+00],
          [ 2.3456e-01,  1.9537e+00,  1.5769e-01,  ...,  1.4087e+00,
            1.2904e+00, -2.6715e-01],
          ...,
          [ 5.6467e-01,  1.7992e-01,  1.4905e+00,  ...,  7.7292e-01,
            8.6508e-01,  8.2257e-01],
          [ 3.6003e-01,  1.3207e+00,  1.0240e+00,  ...,  2.8150e-01,
            6.1890e-01,  3.4369e-01],
          [ 1.6209e+00,  2.0949e-01,  1.2307e+00,  ..., -5.9271e-02,
            8.9028e-01,  1.6284e+00]],
 
         [[-9.1576e-02,  5.3377e-01,  1.7418e-01,  ...,  8.6056e-01,
            1.2178e+00,  9.7631e-01],
          [ 4.4572e-01,  1.2215e+00,  5.0402e-01,  ...,  3.6234e-01,
            8.9612e-01,  4.6493e-02],
          [ 8.4660e-01,  1.9829e+00,  5.7429e-01,  ...,  1.3984e-02,
            1.5751e+00,  1.4441e+00],
          ...,
    

# PyTorch and NumPy Differences

So far we have seen that PyTorch tensors and NumPy arrays are very similar. However, there are some important differences between them.

Computation of gradients of operations on tensors is done automatically, whereas in NumPy we have to do it manually.

$$ y = \sum_{i=1}^n x_i^3 $$ 

has a gradient of

$$ \displaystyle\frac{\partial y}{\partial x_i} = 3 x_i^2 $$

In [23]:
x = torch.tensor([[5., 8.], [4., 6.]], requires_grad=True)

In [24]:
y = x.pow(3).sum()
y

tensor(917., grad_fn=<SumBackward0>)

Compute the gradient:

In [25]:
y.backward()
x.grad

tensor([[ 75., 192.],
        [ 48., 108.]])

Double check the gradient using the analytical formula:

In [26]:
3 * x**2

tensor([[ 75., 192.],
        [ 48., 108.]], grad_fn=<MulBackward0>)

The automatic computation of gradients is very useful, as it allows us to easily implement many machine learning algorithms. The example above is a very simple one, but in practice we often have to compute the gradient of a very complex function, and it would be very tedious to do it manually. 

In general, by having 

$$ y = f(\vec{x}) $$

**PyTorch** can compute $\partial y / \partial \vec{x_i}$ automatically for each $i$ in $\vec{x}$.  

In very simple terms, a **neural network** is a function that takes an input and produces an output. The parameters of the **function** $f$ are called **weights**, stored in a tensor $\vec{x}$. Gradients measure the change of these weights and it's important to compute them automatically, as it allows us to update the weights in the right direction, so that the function $f$ can be optimized.

# Additional Benefits

Another great benefit of **PyTorch** is that it calculates large matrix multiplications even more efficiently than **NumPy** does. This is even more accentuated when using a GPU, which is the main reason why **PyTorch** is so popular in the field of deep learning.

Let's see a quick trivial benchmark of matrix multiplication between **NumPy** and **PyTorch**. 

This is a very simple example, but it shows the difference in speed between the two libraries. The results are even more dramatic as I'm using PyTorch on an *Nvidia RTX 2070 Super* GPU, with **CUDA 11.7**.

## PyTorch example

In [27]:
torch.cuda.is_available()

True

In [39]:
N = int(1e4)
A = torch.randn((N, N))
B = torch.randn((N, N))

In [40]:
t1 = time.perf_counter()
torch.matmul(A, B)
t2 = time.perf_counter()

print(t2 - t1)

2.979303668000057


This is achieved with a CPU computation. Now let's see how much faster it is when using a GPU.

In [41]:
A = A.cuda()
B = B.cuda()

In [42]:
t1 = time.perf_counter()
torch.matmul(A, B)
t2 = time.perf_counter()

print(t2 - t1)

1.023646036999935


## NumPy example

In [34]:
N = int(1e4)
A = np.random.randn(N**2).reshape((N, N))
B = np.random.randn(N**2).reshape((N, N))

In [35]:
t1 = time.perf_counter()
A@B
t2 = time.perf_counter()

print(t2 - t1)

4.56590535700002


The result is quite remarkable, despite my GPU being a mid-range consumer card and my CPU being a 12 core Ryzen 9 3900X. The test could be repeated with larger matrices, but going up a whole order of magnitude would require even more memory than my 64GB of RAM, which is already quite a lot.

One thing I've noticed is that **NumPy** warns you beforehand if you're going to run out of memory, whereas **PyTorch** doesn't. This is a very important feature, as it allows you to avoid running out of memory and crashing your program or your system.

PyTorch actually uses more memory than NumPy in order to be able to compute the operations faster, which is why I did not generate them on the GPU directly, just in case.